In [ ]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2

In [ ]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [ ]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [ ]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [ ]:
print(df_train.shape,df_test.shape)

In [ ]:
df_train.head()

In [ ]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [ ]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [ ]:
print(len(valid_filenames))

valid_filenames = [i for i in valid_filenames if i !='017400.ogg']

print(len(valid_filenames))

In [ ]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [ ]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [ ]:
print(df_train.shape,df_test.shape)

### Upsample minority classes

In [ ]:
df_train['genre_id'].value_counts()

In [ ]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [ ]:
df_train.tail()#,df_train.shape

In [ ]:
df_train['genre_id'].value_counts()

### Create Folds

In [ ]:
df_train['fold'] = 0

In [ ]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [ ]:
df_train.head()

In [ ]:
df_test.head()

#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

spec_tfms = Compose(
    [torchvision.transforms.CenterCrop((160,1072))
    ]
)

cqt_tfms = Compose(
    [torchvision.transforms.CenterCrop((64,960))
    ]
)

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 spec_tfms = spec_tfms,
                 cqt_tfms =cqt_tfms
                ):

        
        self.df = df
        self.train = train
        self.spec_transform = spec_tfms
        self.cqt_transform = cqt_tfms
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/train/{filename}')
            cqt = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/cqt/512_64/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/test/{filename}')
            cqt = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/cqt/512_64/test/{filename}')
        return spec,cqt


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        spec,cqt = self.load_file(row.filename.split('.')[0]+'.npy')
        spec = torch.from_numpy(spec).float()
        cqt = torch.from_numpy(cqt).float()
        
        if self.spec_transform is not None:
            spec = self.spec_transform(spec)
            
        if self.cqt_transform is not None:            
            cqt = self.cqt_transform(cqt)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return spec,cqt, label
        else:
            return spec,cqt

In [ ]:
a,b,c = next(iter(POGFiles(df_train,spec_tfms=None,cqt_tfms=None)))
a.shape,b.shape

In [ ]:
a,b,c = next(iter(POGFiles(df_train,spec_tfms=spec_tfms,cqt_tfms=cqt_tfms)))
a.shape,b.shape

#### FastAI model

In [ ]:
import timm
timm.list_models()

In [ ]:
# timm.create_model('ecaresnext50t_32x4d')

In [ ]:
NCLASS = df_train['genre_id'].nunique()
NCLASS

In [ ]:
class net(nn.Module):
    def __init__(self,in_channels=1,num_classes=NCLASS):
        super(net, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        
        self.net = timm.create_model('ecaresnext50t_32x4d',in_chans=1,pretrained=False)
        self.net. conv1[0].stride = (1,1)
        self.net.fc = nn.Linear(2048,128)
        self.out = nn.Linear(256,self.num_classes)
    def forward(self,spec,cqt):
        spec_pred = self.net(spec)
        cqt_pred = self.net(cqt)
        cat = torch.cat((spec_pred,cqt_pred),1)
#         print(cat.shape)
        out = self.out(cat)
        return out

In [ ]:
class CombinedLoss:
    "Dice and Focal combined"
    def __init__(self, axis=1, smooth=1., alpha=0.5):
        store_attr()
        self.focal_loss = CrossEntropyLossFlat()
        self.dice_loss =  BCEWithLogitsLossFlat()
        
    def __call__(self, spec_pred, cqt_pred,targ):
        pred = (spec_pred, cqt_pred)
        loss = self.focal_loss(pred[0], targ)*(1-self.alpha) + self.alpha * self.dice_loss(pred[0],pred[1] )
        return loss
    
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

In [ ]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'exp_contrastive'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #CombinedLoss()# #BiTemperedLogisticLoss()#CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 24
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            spec_tfms = spec_tfms,
                            cqt_tfms =cqt_tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            spec_tfms = spec_tfms,
                            cqt_tfms =cqt_tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        model = net()
#         model = timm.create_model('convnext_base',in_chans=1,pretrained=False)
# #         model.conv1[0].stride = (1,1)
#         model.fc = nn.Linear(1024,NCLASS)
        
        f1_score = F1Score(average="micro")
        fname = f'{kernel_type}_{fold_num}'
        opt = ranger
        learn = Learner(dls, model,opt_func=opt,loss_func=crit,metrics=[f1_score],
                        model_dir = Path(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}'),
                        cbs=[SaveModelCallback('f1_score',fname=fname, comp=np.greater),
                         CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()

        learn.fit_flat_cos(n_epochs, 1e-2,wd=1e-3)        
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                          spec_tfms = spec_tfms,
                           cqt_tfms =cqt_tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
df_test_valid.head()

In [ ]:
if __name__=='__main__':
    run()

### Mode based sub

In [ ]:
kernel_type

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### OOF

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'ecaresnext50t_32x4d_CQT_448_84'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
# if __name__=='__main__':
#     runTune()

### End ###